In [1]:
pip install PublicDataReader --upgrade

In [67]:
import PublicDataReader as pdr
import pandas as pd

In [85]:
noin_data = pd.read_csv("../data/native_admdong_sum_2023061.csv")

In [92]:
busstop = pd.read_csv("../data/test.csv")
busstop["읍명동인구수"] = 0

In [93]:
연결정보 = pdr.code_hdong_bdong()
연결정보 = 연결정보[연결정보["시도명"] == '경상남도']
연결정보 = 연결정보[연결정보["말소일자"].apply(lambda x: len(x) == 0)]

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.2.1. 시행)


### 주소를 법정동 코드 -> 행정동 코드로 변환 1번 방법



In [96]:
p = 0
for q in range(len(busstop["도시코드"].unique())):
    연결정보_set = 연결정보[연결정보["시군구코드"] == q]
    busstop_set = busstop[busstop["도시코드"] == q]
    for i in range(len(busstop_set["상세주소"])):
        for j in range(len(연결정보_set["동리명"])):
            if 연결정보_set["동리명"][j] in busstop_set["상세주소"][i]:
                for k in range(len(noin_data["admdong_cd"])):
                    if str(noin_data["admdong_cd"][k]) in str(연결정보_set["행정동코드"][j]):
                        busstop["읍명동인구수"][i + p] = noin_data["0"][k]
                        break
                break
    p += len(busstop_set["상세주소"])


In [130]:
busstop

,Unnamed: 0,index_left,정류장번호,정류장명,위도,경도,상세주소,읍명동인구수,도시코드
0,0,10276,CWB379000566,구암고등학교,35.246296,128.593978,경상남도 창원시 마산회원구 구암동,0,38010
1,1,10277,CWB379000567,사격장입구,35.244923,128.683156,경상남도 창원시 의창구 사림동,0,38010
2,2,10278,CWB379000568,장애인복지회관입구,35.244960,128.673597,경상남도 창원시 의창구 봉곡동,0,38010
3,3,10279,CWB379000570,장애인복지회관입구,35.244773,128.673243,경상남도 창원시 의창구 봉곡동,0,38010
4,4,10280,CWB379000573,지귀상가,35.244778,128.659437,경상남도 창원시 의창구 봉곡동,0,38010
...,...,...,...,...,...,...,...,...,...
95,95,10371,CWB379000683,팔용가구거리,35.237389,128.633508,경상남도 창원시 의창구 팔용동,0,38010
96,96,10372,CWB379000914,무학여고,35.221680,128.573473,경상남도 창원시 마산회원구 회원동,0,38010
97,97,10373,CWB379000915,노인병원,35.221711,128.564653,경상남도 창원시 마산회원구 회원동,0,38010
98,98,10374,CWB379000916,셰풀러코리아,35.220529,128.652221,경상남도 창원시 성산구 내동,0,38010


In [127]:
연결정보_set = 연결정보[연결정보["시군구코드"] == "48129"]
연결정보_set

,시도코드,시도명,시군구코드,시군구명,행정동코드,읍면동명,법정동코드,동리명,생성일자,말소일자
45778,48,경상남도,48129,창원시 진해구,4812900000,,4812900000,창원시진해구,20100701,
45813,48,경상남도,48129,창원시 진해구,4812953000,충무동,4812910100,동상동,20200101,
45814,48,경상남도,48129,창원시 진해구,4812953000,충무동,4812910200,도천동,20200101,
45815,48,경상남도,48129,창원시 진해구,4812953000,충무동,4812910300,도만동,20200101,
45816,48,경상남도,48129,창원시 진해구,4812953000,충무동,4812910400,신흥동,20200101,
...,...,...,...,...,...,...,...,...,...,...
45890,48,경상남도,48129,창원시 진해구,4812964000,웅동1동,4812916100,두동,20100701,
45891,48,경상남도,48129,창원시 진해구,4812965000,웅동2동,4812916200,청안동,20100701,
45892,48,경상남도,48129,창원시 진해구,4812965000,웅동2동,4812916300,안골동,20100701,
45893,48,경상남도,48129,창원시 진해구,4812965000,웅동2동,4812916400,용원동,20100701,


In [128]:
list_city =['48120', '48121', '48123', '48125', '48127', '48129',
       '48170', '48220', '48240', '48250', '48270',
       '48310', '48330', '48720', '48730', '48740', '48820',
       '48840', '48850', '48860', '48870', '48880', '48890']